One idea put forward is to vary exposure time in order to keep constant depth. What range of exposure times would be needed?

In [2]:
# Check range of delivered m5 values.
import lsst.sims.maf.db as db

In [4]:
opsdb = db.OpsimDatabase('db/kraken_2026.db')

In [12]:
for f in ('u', 'g', 'r', 'i', 'z', 'y'):
    simdata = opsdb.fetchMetricData(['fiveSigmaDepth'], sqlconstraint='filter="%s"' % f)
    print(f, simdata['fiveSigmaDepth'].min(), simdata['fiveSigmaDepth'].max(), 
          simdata['fiveSigmaDepth'].max() - simdata['fiveSigmaDepth'].min())

u 21.6014854524 24.0837842178 2.4822987654
g 22.8128084211 25.4036678373 2.59085941619
r 22.4302863843 24.9637526347 2.53346625048
i 21.9573319547 24.4080723472 2.45074039248
z 20.8779608031 23.7375564311 2.85959562795
y 20.0817199477 22.735065822 2.65334587434


In [3]:
# Calculate how exposure time should vary. 
import lsst.syseng.throughputs as st

In [6]:
defaultDirs = st.setDefaultDirs()
hardware, system = st.buildHardwareAndSystem(defaultDirs)

In [17]:
# Calculate 
st.makeM5(hardware, system, darksky=None, exptime=30, nexp=1, X=1.0)

,FWHMeff,FWHMgeom,skyMag,skyCounts,Zp_t,Tb,Sb,kAtm,gamma,Cm,dCm_infinity,dCm_double,m5,sourceCounts,m5_fid,m5_min
u,0.92,0.80824,22.989546,70.402589,26.900947,0.032437,0.051867,0.509618,0.037738,23.281862,0.409256,0.246092,23.979911,442.115544,23.9,23.4
g,0.87,0.76714,22.255978,406.358769,28.385271,0.127284,0.152329,0.195029,0.038644,24.517163,0.099552,0.049917,24.909100,737.207358,25.0,24.6
r,0.83,0.73426,21.197245,812.576131,28.169626,0.104355,0.114880,0.104326,0.038945,24.507227,0.052338,0.024335,24.420899,947.571405,24.7,24.3
i,0.80,0.70960,20.476708,1144.015607,27.851170,0.077827,0.083291,0.073665,0.039063,24.379825,0.037756,0.016734,23.973200,1067.356394,24.0,23.6
z,0.78,0.69316,19.599918,1715.474293,27.440200,0.053302,0.055697,0.047725,0.039205,24.201710,0.025521,0.010600,23.384178,1257.564036,23.3,22.9
y,0.76,0.67672,18.611865,2283.488487,26.647213,0.025677,0.029842,0.163195,0.039288,23.754929,0.019306,0.007553,22.471573,1404.033432,22.1,21.7


In [18]:
st.makeM5(hardware, system, darksky=None, exptime=30, nexp=1, X=1.0).m5

u    23.979911
g    24.909100
r    24.420899
i    23.973200
z    23.384178
y    22.471573
Name: m5, dtype: float64

In [34]:
st.makeM5(hardware, system, darksky=None, exptime=30, nexp=4, X=1.0).m5

u    24.748284
g    25.671039
r    25.180731
i    24.732208
z    24.142203
y    23.229024
Name: m5, dtype: float64

In [40]:
st.makeM5(hardware, system, darksky=None, exptime=30, nexp=7, X=1.0).m5 - \
st.makeM5(hardware, system, darksky=None, exptime=30, nexp=1, X=1.0).m5

u    1.076027
g    1.068022
r    1.065401
i    1.064376
z    1.063153
y    1.062439
Name: m5, dtype: float64

To account for a magnitude of variation, you would need 7x30s exposures (vs. just 1x30s exposure).

In [44]:
st.makeM5(hardware, system, darksky=None, exptime=30, nexp=120, X=1.0).m5 - \
st.makeM5(hardware, system, darksky=None, exptime=30, nexp=1, X=1.0).m5

u    2.627691
g    2.615996
r    2.612166
i    2.610669
z    2.608882
y    2.607840
Name: m5, dtype: float64

To account for just slightly over 2.6 magnitudes of variation, you would need a whopping 120x30s exposures. 